# Part 03: Find the discontinuities in Seattle's streets
michael babb  
2024 11 24

In [1]:
# standard
import os

In [2]:
# external
import geopandas as gpd
import networkx as nx
import numpy as np
import pandas as pd
from shapely.geometry import LineString, Point

In [3]:
# custom
from geodataio.geo_operations import points2distance
from utils import *

# load the working seattle street network data

In [4]:
# file path
input_file_path = 'H:/project/seattle_streets/data/' 
output_file_path = 'H:/project/seattle_streets/data/'


In [5]:
file_name = 'Street_Network_Database_Seattle_working.gpkg'

In [6]:
fpn = os.path.join(input_file_path, file_name)

In [7]:
gdf = gpd.read_file(filename = fpn)

# load the node data

In [8]:
input_file_name = 'Street_Network_Nodes.gpkg'

In [9]:
fpn = os.path.join(input_file_path, input_file_name)

In [10]:
node_gdf = gpd.read_file(filename = fpn)

In [11]:
node_gdf.head()

,node_id,geometry
0,17740,POINT (-122.32287 47.52982)
1,11391,POINT (-122.32402 47.61849)
2,982,POINT (-122.29193 47.7649)
3,11695,POINT (-122.30782 47.61411)
4,6257,POINT (-122.38214 47.67456)


In [12]:
node_gdf['node_id'] = node_gdf['node_id'].astype(int)

In [13]:
node_gdf['coords'] = node_gdf['geometry'].map(lambda x: x.coords[0])

In [14]:
node_gdf.head()

,node_id,geometry,coords
0,17740,POINT (-122.32287 47.52982),"(-122.32287444824671, 47.529819957875)"
1,11391,POINT (-122.32402 47.61849),"(-122.32401964037119, 47.61848906651073)"
2,982,POINT (-122.29193 47.7649),"(-122.29192836110747, 47.764904310419276)"
3,11695,POINT (-122.30782 47.61411),"(-122.30781531879083, 47.61410595110372)"
4,6257,POINT (-122.38214 47.67456),"(-122.38214200036303, 47.67455801597557)"


In [15]:
node_gdf.dtypes

node_id        int32
geometry    geometry
coords        object
dtype: object

In [16]:
# zap this into a dictionary
node_dict = {}
for i, row in node_gdf.iterrows():
    node_dict[row['node_id']] = row['coords']

# find missing segments

In [17]:
# create a weight variable from the gis_segment_length variable
gdf['weight'] = gdf['gis_seg_length']

In [18]:
# only select columns names used in subsequent steps
col_names = ['f_intr_id',
't_intr_id',
'snd_id',
'snd_feacode',
'gis_seg_length',
'ord_street_name',
'ord_street_type',
'ord_stname_concat',
'city_portion',
'geometry',
'weight']

In [19]:
gdf = gdf[col_names].copy()

In [20]:
# what's the current max snd ID?
gdf['snd_id'].max()

50338

In [21]:
2**16

65536

In [36]:
# here is where the discontinuities are identified and the "missing" segments are created.
# a full run, without saving intermiedate data, takes less than a minute.
# saving intermediate data takes an additional ~5 minutes.

# save intermediate data for checking?

write_intermediate = True

# street_status
# 0: complete street
# 1: disconnected
# 2: missing street

# hold the output
output_gdf_list = []
shortest_added_path = []
gdf['snd_group'] = int(0)
snd_group_count = -1
# use this value to start the ID number for the added segements.
temp_snd_id = 2**16
# get the list of unique names
s_name = gdf['ord_stname_concat'].unique().tolist()
# uncomment to test
#s_name = ['7TH AVE N', 'W GALER ST']
#s_name = ['W GALER ST']
s_name = ['15TH AVE W']

# the trouble_list, for lack of a beter name, are streets with parallel edges
# we can record the names of these streets. 
trouble_list = []

for sn in s_name[:None]:
    print(sn)
    
    # subset the gdf by streetname
    temp_gdf = gdf.loc[gdf['ord_stname_concat'] == sn, :].copy().reset_index(drop = True)
    
    # create the graph for a single street.
    # For example, all streets names W GALER ST are a single graph.
    # this is an undirected graph.
    fg = nx.from_pandas_edgelist(df = temp_gdf, source = 'f_intr_id', target = 't_intr_id', edge_attr=True)
    
    if temp_gdf.shape[0] != fg.number_of_edges():
        trouble_list.append(sn)            
        
        # subsequent steps need a single-edge graphs. 
        # Parallel edges are not allowed. 
        # so, let's return the edges last added to the graph.
        mod_edge_list = []
        for ed in fg.edges.data():
            mod_edge_list.append(ed[2]['snd_id'])
        temp_gdf = temp_gdf.loc[temp_gdf['snd_id'].isin(mod_edge_list), :]
        
    # get the list of nodes in the graph.
    node_list = list(fg.nodes)
    
    # a list of the snd groups - this is necessary to match street segments to nodes
    # nx.connected_components() identifies disconnected graphs
    for cc in nx.connected_components(G = fg):
        # cc is the sub graph
        sub_graph = fg.subgraph(cc).copy()
        # this is the edge data        
        edge_data_list = sub_graph.edges.data()        
        # this marks the street(s) in each sub graph
        snd_id_list = []
        for edl in edge_data_list:                        
            snd_id_list.append(edl[2]['snd_id'])        
        snd_group_count += 1
        # this will update the gdf per street name with the groups of streets
        temp_gdf.loc[temp_gdf['snd_id'].isin(snd_id_list), 'snd_group'] = snd_group_count     
    
    # this is the number of unique snd_groups
    # also the number of discontinuities
    snd_group_id_list = temp_gdf['snd_group'].unique()    
    
    n_discontinuities = len(snd_group_id_list)    
    
    if n_discontinuities > 1:
        # process for the sub graphs
        node_snd_group_dict = {}
        # a node - an intersection - can have more than one street.
        # we need to update the node snd group dict for every street
        # identify each edge that each node is on
        for ir, row in temp_gdf.iterrows():       
            fn = row['f_intr_id']
            tn = row['t_intr_id']
            snd_group_id = row['snd_group']
            node_snd_group_dict[fn] = snd_group_id
            node_snd_group_dict[tn] = snd_group_id                                             
        
        # create a list of available edges - these are the missing segments
        # these are formed from the non-edges of the graph.        
        avail_edges = []

        # dictionary to hold available edges
        # let's only make the distance calculation once, yeah?
        # it's a simple calculation, but even more simple to store it. 
        node_dist_dict = {}
        # enumerate the non-edges
        non_edge_count = 0
        # the total number of non-edges
        diff_non_edge_count = 0
        # the number of non-edges that connect disconnected components
        for ne in nx.non_edges(graph = fg):    
            non_edge_count += 1
            # ne is a tuple of from / to nodes.
            # create available edges if the nodes are not on the same segment
            # this will decrease the potential solution space
            if node_snd_group_dict[ne[0]] != node_snd_group_dict[ne[1]]:
                diff_non_edge_count += 1
                # calculate the straight-line distance between two nodes. 
                # convert to feet to match the existing distance / weight variable
                weight = points2distance(node_dict[ne[0]], node_dict[ne[1]], unit = 'miles') * 5280
                # build the output tuple
                output = (ne[0], ne[1], {'weight':weight})
                # add to the distance dict
                node_dist_dict[(ne[0], ne[1])] = weight 
                node_dist_dict[(ne[1], ne[0])] = weight 
                avail_edges.append(output)

        # we just calculated all of the potential missing segments that span discontinuities
        # nx.k_edge_augmentation() creates the missing edges in a graph by adding as
        # few edges as possible. In a street network, with multiple disconnected
        # components, there is really one way to minimally connect the disparate components
        # to create full connectivity. And it's the shortest geographic segment in this case!
        # weighted graph traversal works by accumulating as little weight as possible. 
        
        # this will store the output
        data_list = []
        line_list = []
        # once nx.k_edge_augmentation() finishes, it returns a generator with the added edges that
        # ensure complete connectivity between all nodes.
        augmented_edges = nx.k_edge_augmentation(G = fg, k = 1, avail = avail_edges, weight = 'weight')    
        # enumerate the augmented edges
        for i_ae, ae in enumerate(augmented_edges):
            # unpack
            fn, tn = ae            
            # get the weight / distance of the added edge
            weight = node_dist_dict[(fn, tn)] 
            # this is the output dictionary
            # street name, integer indicating the snd_group, from node, to node, distance of the edge
            temp_snd_id += 1
            temp_data_list = [temp_snd_id, sn, i_ae, fn, tn, weight]
            # now, let's create some geometry
            temp_line = LineString([node_dict[fn], node_dict[tn]])
            line_list.append(temp_line)
            data_list.append(temp_data_list)

        # compare the list of available edges to the list of chosen edges
        # this isn't necessary to complete the edge augmentation, but it's nice to know how often
        # added edges are always the shortest edges        
        av_df = pd.DataFrame(data = avail_edges, columns = ['sn_id', 'en_id', 'weight_dict'])
        av_df['dist'] = av_df['weight_dict'].map(lambda x: x['weight'])
        av_df = av_df.drop(labels = 'weight_dict', axis = 1)
        # rank distance - the lower the rank, the shorter the segment.
        av_df['dist_rank'] = av_df['dist'].rank(method = 'dense')
        
        # build a gpd.GeoDataFrame - these are the "missing" segments"
        ms_gdf = gpd.GeoDataFrame(data = data_list,
                                         columns = ['snd_id', 'ord_stname_concat', 'snd_group', 'sn_id', 'en_id', 'dist'],
                                         geometry = line_list, crs = 'epsg:4326')
        # these edges are not on the same street group
        ms_gdf['street_status'] = int(2)

        # this will track if there are any graphs with parallel edges.
        if ms_gdf.empty:
            trouble_list.append(sn)            
        
        # select where the rank is LTE than the number of records in the missing segement gdf
        av_df = av_df.loc[av_df['dist_rank'] <= ms_gdf.shape[0], :]
        
        # if the sum of the distance of the added segments is the same, then only the
        # shortest segments were added. If not, then other, longer, segments were added
        # but that makes for less total distance traversed in the graph.
        # So, how often does that happen.
        shortest_tot_path = av_df['dist'].sum() == ms_gdf['dist'].sum()
        shortest_added_path.append([sn, shortest_tot_path, n_discontinuities - 1])        
                
        # add the known, disconnected segments. These are the existing streets.
        col_names = ['snd_id', 'ord_stname_concat', 'snd_group',  'f_intr_id', 't_intr_id', 'gis_seg_length', 'geometry']    
        ks_gdf = temp_gdf[col_names].copy()
        ks_gdf['street_status'] = int(1)
        ks_gdf = ks_gdf.rename(columns = {'f_intr_id':'sn_id', 't_intr_id':'en_id', 'gis_seg_length':'dist'})

        # stack the geodataframes
        output_gdf = pd.concat([ms_gdf, ks_gdf])
    
        # write intermediate
        if write_intermediate:
            # intermediate streets - existing streets
            is_output_file_path = os.path.join(output_file_path, 'individual_streets')
            output_file_name = '_'.join(sn.split()) + '.gpkg'
            write_gdf(gdf = temp_gdf, output_file_path = is_output_file_path, output_file_name = output_file_name)
        
            # intermediate nodes
            curr_node_df = pd.DataFrame(data = {'node_id':fg.nodes()})
            curr_node_list = curr_node_df['node_id'].tolist()
            node_subset_gdf = subset_node_gdf(node_gdf = node_gdf, other_node_df = curr_node_df)
            output_file_name = '_'.join(sn.split()) + '_full_nodes.gpkg'
            write_gdf(gdf = node_subset_gdf, output_file_path = is_output_file_path, output_file_name = output_file_name)

            # the missing and known segments for a street
            output_file_name = '_'.join(sn.split()) + '_missing_segments.gpkg'
            ofpn = os.path.join(output_file_path, output_file_name)                
            write_gdf(gdf = output_gdf, output_file_path = is_output_file_path, output_file_name = output_file_name)

    else:
        # gather the streets with no missing segments.         
        col_names = ['snd_id', 'ord_stname_concat', 'snd_group', 'f_intr_id', 't_intr_id', 'gis_seg_length', 'geometry']    
        output_gdf = temp_gdf[col_names].copy()
        output_gdf['street_status'] = int(0)
        output_gdf = output_gdf.rename(columns = {'f_intr_id':'sn_id', 't_intr_id':'en_id', 'gis_seg_length':'dist'})

    # add to the output list
    output_gdf_list.append(output_gdf)

15TH AVE W


# combine data for output

In [37]:
ms_gdf = pd.concat(objs = output_gdf_list)

In [38]:
ms_gdf['dist_miles'] = ms_gdf['dist'] / 5280

In [25]:
# join in other street indentification data

In [26]:
ms_gdf = pd.merge(left = ms_gdf, right = gdf[['ord_stname_concat', 'ord_street_type', 'ord_street_name']].drop_duplicates())

In [27]:
ms_gdf.shape

(27, 11)

In [28]:
ms_gdf['street_status'].value_counts()

street_status
1    24
2     3
Name: count, dtype: int64

In [29]:
# check for missing values
for cn in ms_gdf.columns:
    print(cn, ms_gdf[cn].isna().unique())    

snd_id [False]
ord_stname_concat [False]
snd_group [False]
sn_id [False]
en_id [False]
dist [False]
geometry [False]
street_status [False]
dist_miles [False]
ord_street_type [False]
ord_street_name [False]


In [30]:
# what streets have parallel edges?
# these can be verified / examined in qGIS
trouble_list

[]

# compute if the added segments were indeed the shortest.

In [31]:
# the number of times when the shortest segments
sap_df = pd.DataFrame(data = shortest_added_path, columns = ['ord_stname_concat', 'shortest_added_path', 'n_discontinuities'])

In [32]:
sap_df['n_streets'] = int(1)

In [33]:
sap_df.head()

,ord_stname_concat,shortest_added_path,n_discontinuities,n_streets
0,15TH AVE W,False,3,1


In [34]:
sap_df_ct_count = pd.pivot_table(data = sap_df, values = 'n_streets',
                                 index = 'n_discontinuities', columns = 'shortest_added_path',
                                 aggfunc='sum', fill_value=0, margins = True).reset_index(drop=False)

In [35]:
sap_df_ct_count.columns = ['n_discontinuities', 'shortest_is_false', 'shortest_is_true', 'n_streets']

ValueError: Length mismatch: Expected axis has 3 elements, new values have 4 elements

In [ ]:
sap_df_ct_count.columns

Index(['n_discontinuities', 'shortest_is_false', 'shortest_is_true',
       'n_streets'],
      dtype='object')

In [ ]:
for cn in ['shortest_is_false', 'shortest_is_true']:
    ncn = cn + '_per'
    sap_df_ct_count[ncn] = sap_df_ct_count[cn] / sap_df_ct_count['n_streets']

In [ ]:
sap_df_ct_count.head()

,n_discontinuities,shortest_is_false,shortest_is_true,n_streets,shortest_is_false_per,shortest_is_true_per
0,1,0,398,398,0.000000,1.000000
1,2,171,44,215,0.795349,0.204651
2,3,131,8,139,0.942446,0.057554
3,4,108,4,112,0.964286,0.035714
4,5,77,1,78,0.987179,0.012821


In [ ]:
# export this to excel
output_file_name = 'shortest_segment_count.xlsx'
ofpn = os.path.join(output_file_path, 'analysis', output_file_name)
sap_df_ct_count.to_excel(excel_writer=ofpn, sheet_name='shortest_segment_count', index = False)

# add ranking by street type to help with visualization

In [ ]:
st_type_df = ms_gdf.loc[ms_gdf['street_status'] == 2, 'ord_street_type'].value_counts().to_frame(name = 'n_segments').reset_index()

In [ ]:
st_type_df.head()

,ord_street_type,n_segments
0,AVE,1806
1,ST,1661
2,PL,89
3,WAY,34
4,DR,19


In [ ]:
st_type_df['segment_rank'] = st_type_df['n_segments'].rank(ascending = True).astype(int).astype(str).str.zfill(2)

In [ ]:
st_type_df.head(n=20)

,ord_street_type,n_segments,segment_rank
0,AVE,1806,10
1,ST,1661,09
2,PL,89,08
3,WAY,34,07
4,DR,19,06
5,LN,13,04
6,BLVD,13,04
7,RD,5,03
8,CT,2,02
9,PKWY,1,01


In [ ]:
st_type_df['ord_street_type_rank'] = st_type_df['segment_rank'] + '_' + st_type_df['ord_street_type']

In [ ]:
st_rank_dict = {ost:ostr for ost, ostr in zip(st_type_df['ord_street_type'], st_type_df['ord_street_type_rank'])}

In [ ]:
ms_gdf['ord_street_type_rank'] = ms_gdf['ord_street_type'].map(st_rank_dict)

In [ ]:
ms_gdf.loc[ms_gdf['ord_street_type_rank'].isna(), 'ord_street_type_rank'] = ""

In [ ]:
# reorder columns
col_names = ['snd_id', 'ord_street_name','ord_stname_concat','ord_street_type', 
             'ord_street_type_rank', 'snd_group', 'street_status','sn_id',
             'en_id','dist','dist_miles','geometry']

In [ ]:
ms_gdf = ms_gdf[col_names]

# save the geodataframe with the complete, disconnected, and added streets

In [ ]:
output_file_name = 'missing_segments.gpkg'
ofpn = os.path.join(output_file_path, output_file_name)    

ms_gdf.to_file(filename = ofpn, driver = 'GPKG', index = False)
